<a href="https://colab.research.google.com/github/jonghhhh/lecture_colabs/blob/main/%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%9C%A0%EC%82%AC%EB%8F%84_%EA%B2%80%EC%83%89%EA%B3%BC_%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81_CLIP_ViT_FAISS_K_means_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지 유사도 검색과 클러스터링: CLIP, ViT, FAISS, K-means 활용


### CLIP (Contrastive Language–Image Pre-Training)
- OPEN AI의 이미지와 텍스트를 함께 학습하는 모델로, 자연어와 시각적 표현을 결합하여 강력한 인식 성능 제공

### ViT-B/32
- Vision Transformer 모델의 변형으로, 이미지 입력을 토큰화하여 트랜스포머 아키텍처에 전달
- B = Base 모델.
- 32 = 이미지가 32x32 픽셀 패치로 분할되어 입력

### FAISS(Facebook AI Similarity Search)
- 대규모 벡터 검색 및 클러스터링을 위한 라이브러리. 주로 고차원 벡터의 빠른 유사성 검색을 수행

### K-means 클러스터링
- 비지도 학습 알고리즘으로, 데이터를 여러 그룹(클러스터)으로 나누는 데 사용

In [ ]:
# 패키지 설치
!pip install ftfy regex tqdm   # 아래 CLIP 전처리에 필요. ftfy: 텍스트를 깨끗하게 수정
!pip install git+https://github.com/openai/CLIP.git
!pip install faiss-gpu
!pip install torchvision
!pip install numpy
!pip install pillow

## 검색: 전체 데이터에서 유사한 이미지 찾기(코사인유사도)

In [ ]:
import torch
import clip
from PIL import Image, ImageOps
import faiss
import numpy as np
import os
from sklearn.manifold import TSNE
import pandas as pd
import pickle
from scipy.spatial.distance import cdist

# CLIP 모델 로드 및 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device) # CLIP 모델과 해당 전처리 함수를 로드. preprocess: 이미지 전처리.

# 이미지 벡터화 함수
def get_image_vector(image_path, model, preprocess):
    image = Image.open(image_path).convert('RGB')
    image = ImageOps.grayscale(image)  # 이미지 흑백 변환
    image = image.convert('RGB')  # 모델이 RGB 입력을 기대하므로 다시 RGB로 변환
    image = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    vector = image_features.cpu().numpy().squeeze()
    return vector

# 이미지 폴더 경로 설정
image_folder = '/--image_folder/'

# 이미지 벡터와 경로 저장을 위한 리스트
image_vectors = []
image_paths = []

# 이미지 벡터화 및 경로 저장
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename)
        vector = get_image_vector(image_path, model, preprocess)
        image_vectors.append(vector)
        image_paths.append(filename)  # 경로 대신 파일 이름만 저장

# 리스트를 numpy 배열로 변환
image_vectors = np.array(image_vectors).astype(np.float32)

# 벡터 정규화 (L2 정규화) - 코사인 유사도를 위해
faiss.normalize_L2(image_vectors)

# GPU 사용을 위한 설정
res = faiss.StandardGpuResources()
index_flat = faiss.IndexFlatIP(image_vectors.shape[1])  # 코사인 유사도를 위한 Inner Product
gpu_index = faiss.index_cpu_to_gpu(res, 0, index_flat)

# Faiss 인덱스에 데이터 추가
gpu_index.add(image_vectors)

def find_similar_images(query_image_path, top_k=5):  # top_k: 상위 몇 개의 이미지를 반환할지 결정
    # 쿼리 이미지 벡터화
    query_vector = get_image_vector(query_image_path, model, preprocess).astype(np.float32)
    faiss.normalize_L2(query_vector.reshape(1, -1))

    # 코사인 유사도 계산 (내적을 사용하여)
    D, I = gpu_index.search(query_vector.reshape(1, -1), top_k)

    # 유사한 이미지와 유사도 데이터프레임 생성
    similar_images = [image_paths[i] for i in I[0]]
    similarities = 1 - D[0]  # 거리를 유사도로 변환
    df_similar = pd.DataFrame({'image': similar_images, 'cosine_similarity': similarities})

    # 유사도 내림차순으로 정렬
    df_similar = df_similar.sort_values(by='cosine_similarity', ascending=False).reset_index(drop=True)
    return df_similar

# 예제 쿼리 이미지 경로
query_image_path = 'new_image.jpg'

# 유사한 이미지 검색 및 결과 출력
df_similar_images = find_similar_images(query_image_path)
print(df_similar_images)

## 군집분석(clustering)

### K-means에서 적절한 군집 개수 찾기(Elbow Method) 함수화
- 군집(클러스터) 수에 따른 WCSS(Within-Cluster Sum of Squares) 계산해, WCSS가 급격히 감소했다 완화하는 지점 찾기

In [ ]:
!pip install faiss-gpu
!pip install matplotlib

import faiss
import matplotlib.pyplot as plt

def calculate_wcss(document_vectors):
    wcss = []
    for i in range(1, 11):
        kmeans = faiss.Kmeans(d=document_vectors.shape[1], k=i, niter=20, gpu=True)
        kmeans.train(document_vectors)
        wcss.append(kmeans.obj[-1])
    return wcss

### 군집분석: 코사인 유사도 사용

In [ ]:
import torch
import clip
from PIL import Image, ImageOps
import faiss
import numpy as np
import os
from sklearn.manifold import TSNE
import pandas as pd
import pickle


# CLIP 모델 로드 및 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 이미지 벡터화 함수
def get_image_vector(image_path, model, preprocess):
    image = Image.open(image_path).convert('RGB')
    image = ImageOps.grayscale(image)  # 이미지 흑백 변환
    image = image.convert('RGB')  # 모델이 RGB 입력을 기대하므로 다시 RGB로 변환
    image = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    vector = image_features.cpu().numpy().squeeze()
    return vector

# 이미지 폴더 경로 설정
image_folder = '/content/drive/MyDrive/gwanjingu/dance/'

# 이미지 벡터와 경로 저장을 위한 리스트
image_vectors = []
image_paths = []

# 이미지 벡터화 및 경로 저장
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename)
        vector = get_image_vector(image_path, model, preprocess)
        image_vectors.append(vector)
        image_paths.append(filename)  # 경로 대신 파일 이름만 저장

# 리스트를 numpy 배열로 변환
image_vectors = np.array(image_vectors).astype(np.float32)

# 벡터 정규화 (L2 정규화) - 코사인 유사도를 위해
faiss.normalize_L2(image_vectors)

# GPU 사용을 위한 설정
res = faiss.StandardGpuResources()
index_flat = faiss.IndexFlatIP(image_vectors.shape[1])  # 코사인 유사도를 위한 Inner Product
gpu_index = faiss.index_cpu_to_gpu(res, 0, index_flat)

# 클러스터 개수 도출: WCSS 계산
wcss = calculate_wcss(document_vectors)
plt.figure(figsize=(10, 5))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Faiss를 이용한 클러스터링
num_clusters = 4  # 위에서 확인한 최적의 cluster 개수 입력
kmeans = faiss.Kmeans(d=image_vectors.shape[1], k=num_clusters, niter=100, gpu=True, spherical=True)  # spherical=True로 설정하여 클러스터링시 벡터를 정규화
kmeans.train(image_vectors)

# 클러스터 할당
_, cluster_assignments = kmeans.index.search(image_vectors, 1)
cluster_assignments = cluster_assignments.flatten()  # 2D 배열에서 1D 배열로 변환

# 클러스터 중심점 확인과 저장
cluster_centroids = kmeans.centroids
cluster_centroids.shape
with open('cluster_centroids.pkl', 'wb') as f:
    pickle.dump(cluster_centroids, f)

# 클러스터 중심점과의 유사도 계산
similarities = []
for i, vector in enumerate(image_vectors):
    cluster_idx = cluster_assignments[i]
    centroid = cluster_centroids[cluster_idx]
    similarity = 1 - cdist([vector], [centroid], metric='cosine')[0][0]  # 코사인 유사도 계산
    similarities.append(similarity)

# 클러스터 결과를 DataFrame으로 변환
df_clustered = pd.DataFrame({'image': image_paths, 'cluster': cluster_assignments, 'cosine_similarity_to_centroid': similarities})
df_clustered

## 시각화: t-SNE(t-distributed Stochastic Neighbor Embedding) 활용

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE


# TSNE 변환
data_with_centroids = np.vstack([image_vectors, cluster_centroids])
perplexity = min(30, len(image_vectors) - 1)  # perplexity 값을 샘플 수보다 작게 설정
tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42)
tsne_results = tsne.fit_transform(data_with_centroids)

# TSNE 결과를 DataFrame으로 변환
df_tsne = pd.DataFrame(tsne_results[:len(image_vectors)], columns=['x', 'y'])
df_tsne['image'] = image_paths
df_tsne['cluster'] = cluster_assignments

# TSNE 결과 시각화
plt.figure(figsize=(10, 8))
for cluster in range(num_clusters):
    clustered = df_tsne[df_tsne['cluster'] == cluster]
    plt.scatter(clustered['x'], clustered['y'], label=f'Cluster {cluster}')

# 중심점 시각화
centroid_tsne = tsne_results[len(image_vectors):]
for i, centroid in enumerate(centroid_tsne):
    plt.scatter(centroid[0], centroid[1], marker='X', s=200, label=f'Centroid {i}', edgecolor='black')

plt.legend()
plt.title('t-SNE Results')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()